Below is the code for scraping tournament data. We will likely convert this to a script.

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from urllib.request import urlretrieve
import os

`get_link` gets all links to games where fox is playing vs jigglypuff. We will eliminate this requirement when we go more general.

In [2]:
def get_link(g):
    # Get the image icons of the characters
    icons = g.find_elements_by_css_selector("img")
    # Get the icon ids based on the source image. This shows which characters are being played.
    icon_ids = sorted([int(icon.get_attribute("src").split("icon")[1].split("-")[1]) for icon in icons])
    # Return None if it's not Fox vs Puff
    if icon_ids != [2, 15]:
        return None
    # Else return the link to the game.
    return g.find_element_by_css_selector("a").get_attribute("href")

Starts the selenium web driver.

In [3]:
driver = webdriver.Firefox()

The main code: goes through each tournament page and gets all links.

In [4]:
links = []
for i in range(12):
    driver.get("https://slippi.gg/tournament/97437?page={}".format(i+1))
    delay = 10
    # Waits 10 seconds for page to load
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))
    except TimeoutException:
        print("Loading took too much time!")
    # Gets the games from the table, and thus the links.
    games = driver.find_element_by_css_selector("tbody").find_elements_by_css_selector("tr")
    cur_links = [get_link(game) for game in games]
    cur_links = [link for link in cur_links if link != None]
    links += cur_links

Downloads all the links that were retrieved.

In [72]:
for link in links:
    urlretrieve(link, os.path.join("./slip", link.split("/")[-1]))